# AMLD workshop by L2F – Learn to Forecast

Welcome to the "Machine Learning Competition: Tennis Analysis" workshop at AMLD 2019, organized by [L2F – Learn to Forecast](https://www.l2f.ch/)!

Starting from Jeff Sackman's crowdsourced [Match Charting Project](https://github.com/JeffSackmann/tennis_MatchChartingProject) dataset, we have put together for you a dataset containing detailed shot-by-shot descriptions of a large collection of points played between the Swiss tennis star Roger Federer and his historical rival Rafael Nadal (RN). The information recorded includes the type of shot, direction of shot, depth of returns, types of errors, and more. This is better explained below; we have also put together some preliminary data preparation to help you get started.

## Goal

The goal of this workshop is for you to come up with an optimized Phederos – the god of tennis – who will be able to destroy his ancient human rival. Do you think you can beat the beast?

From a more technical perspective, we ask you to provide a strategy function which, given an opponent's shot, will return what you believe to be an optimal response shot. Your answer must be in the form of a JSON dictionary as follows: each key is an opponent's shot, and the corresponding value is a list from which we will uniformly sample your response.

For more clarity, let us consider an example of such a strategy (the strings denote shot types which are better explained below):

*strat* = {'f3': ['b2', 'b2', 'b3', 'v1'], 'f1': ['f2'], ...}.

The keys in this dictionary are shots RN can produce. Suppose RN is performing a forehand on your backhand (the code for this shot is 'f3'); *strat* would reply to this shot in a *probabilistic* way as follows: 
- 'b2' with probability 1/2,
- 'b3' with probability 1/4,
- 'v1' with probability 1/4.

On the other hand, the same strategy replies *deterministically* to 'f1' (which is a forehand on our forehand), since we say 'f2' (a forehand straight to RN) with probability 1. In this way, your strategy can incorporate simple deterministic and more complex probabilistic decision-making in an arbitrary way.

## Testing environment

You have to beat a stochastic model which replies to your shots based on past statistics, but never replies with shots which have historically occurred too rarely. More precisely, this simulation environment will react based on the counting statistics of outcomes to (human) Federer's shots. By 'outcome' here we mean either:
- a simple reply shot or serve by historical RN (this is perhaps a bit confusing at first, but we consider a player's serve as a 'reply' to the opponent's 'waiting');
- a 'termination token', e.g. '*' for a winning shot, meaning that RN could not respond to your winning shot and the point terminates, or '@' indicating that with your shot you made an unforced error and lost the point – but note that such tokens can include more detailed information than this and be more than one character long;
- a shot by RN and a termination token, concatenated in a simple string, e.g., 'f1*' would indicate that our simulated RN replied with a winning forehand shot – ouch!

The Python code for this tester can be found in the last part of this notebook.

### Important remarks

- The simulation environment will prompt you with shots or termination/waiting tokens from a predetermined list, which can be accessed by loading the pickle file *possible_prompts.p*. Make sure you know how to react to each of the elements in this list!
- The pickle file *allowed_shots.p* contains a list which can be obtained from the list in *possible_prompts.p* by removing termination and waiting tokens. Replying with a string not in this list leads to losing of the point. 
- The "past statistics" in the stochastic model you will be fighting against come from a hold-out "test set" and **not** from the dataset you have access to. Make sure to build models which generalize well!
- We will impose a time constraint on the duration of the evalutation of your policy: we expect that your policy can be evaluated on an i7 quadcore 1000000 times in less than 3 minutes.

Will the true heroes of Machine Learning rise? :)

# Data from Jeff Sackmann's GitHub

As explained above, we provide you with a pandas dataframe (*dataframe.p*), taken from Jeff Sackmann's original dataset. 

Each row is one point:
- the variables 'Player1' and 'Player2' are the name of the players
- the 'match_id' variable identifies the tournament and date
- the varaible 'Shots' is a string containing all the shots, played alternatively from the players, during the point
- the variable 'Shots1' are the shots played only by RF
- the variable 'Shots0' contain the shots played only by RN
- The 'Outcome' variable describes the outcome of the point
- The 'Server' variable has boolean values: it is 1 in case RF is serving
- The 'Winner' variable has boolean values: it is 1 in case RF has won the point

Numbers are used to indicate direction and depth, while letters are used to specific shot types (e.g. 'f' stands for 'forehand') and error types ('n' stands for 'net'). The letter 'X' means that RN is waiting for RF to serve... don't let him wait too much! A few symbols are used for other purposes, such as types of errors (e.g. '@' means 'unforced error' and '+' indicates an approach shot).

To understand the codes for each shot, we prepared an explanation file named "Shot encodings" with more details. You can also check [this example video](https://drive.google.com/open?id=1Q4obFagfzxkFy_mq9C5gWITQiXxBQfCf) for a visual demonstration of this way of encoding.

In [3]:
# Importing libraries

import numpy as np
import pickle
import pandas as pd
from nltk import ngrams
from collections import Counter

In [4]:
# Historical data

df = pd.read_pickle('dataframe.p')
df.head(50)

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
1,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b3,b3,6,w#,0,0
2,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6,,6,*,0,0
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1
5,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b2,4,b2,d#,1,1
6,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 f37 s1,4 s1,f37,n#,1,0
7,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b28 b2 s3 f+1 b1,4 b2 f+1,b28 s3 b1,w#,1,1
8,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4,4,,*,1,1
9,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 f37 s1 b1,5 s1,f37 b1,w@,1,1


# Prompts and shots

We provide here the list of possible prompts and allowed shots – historically the most common shots – that you have to use. There are many more shots in the dataframe, but we only stick to the 100 shots provided and the 40 shots provided. It means that the historical RN/tester, will only shoot with one of the 100 prompts and you can reply using only one of the 40 shots.

In [5]:
# Loading the prompts and the shots

all_shots = pickle.load(open("allowed_shots.p", "rb"))
poss_prompts = pickle.load(open("possible_prompts.p" , "rb"))

print('Allowed shots: ', all_shots, '\n')
print('Possible prompts: ', poss_prompts)

len(poss_prompts)


Allowed shots:  ['4', '4+', '5', '6', '6+', 'X', 'b', 'b1', 'b17', 'b18', 'b19', 'b2', 'b27', 'b28', 'b29', 'b3', 'b37', 'b38', 'f', 'f+1', 'f+3', 'f1', 'f18', 'f2', 'f27', 'f28', 'f29', 'f3', 'f37', 'f38', 'r2', 's', 's1', 's2', 's28', 's3', 'v1', 'v3', 'z1', 'z3'] 

Possible prompts:  ['f3', 'f1', 'b2', 'b1', 'X', 'f2', 'b3', '4', '6', '5', '*', 'b28', 'f28', 'b27', 's2', 'f+3', 's3', 'n@', 'f+1', 'b18', 'd@', 'f38', 'b29', 's1', 'b38', 'f37', '#', 'w@', 'f27', 'f29', 'n#', 'd#', 'f', 'b', 's28', 'b37', 'b17', 'f1*', 'b19', 'b39', 'w#', 'r2', 'f18', 'f39', 'z3', 'v1', 'v3', 'f#', 's27', '4+', '6+', 'z1', 'b1*', 'f3*', 'm2', 'f17', 'r3', 's', 'r1', 's29', 's38', 'b+1', 'z2', 'o3', 'c6', 'v2', 'c4', 'b1w@', 'b#', 'b1n@', 'b+3', 'f19', 'b3*', 'f+2', 'f1w@', 'b3w@', 's37', 's39', 'f3n@', 'f3w@', 'm3', 'f1n@', 'b1d@', 'b2n#', 'f2d@', 'f1n#', 'b1w#', 's#', 'b1n#', 'x@', 'f1w#', 'y3', 'f2d#', 'b2d@', 'f2n#', 'f3d@', 'y1', 'b3w#', 'o1', 'f-3']


100

# Testing function for your model performances

See the file *aux_functions*.  The following function makes the historical RN become alive and play against you!

In [6]:
from aux_functions import *

In [7]:
Model, shots_opo_com = aux_fn(df, poss_prompts, all_shots)
Model

array([[list([]), list([]), list([]), ..., list([]),
        list(['m3', '*', 'b3*', '*', 'b1n#', 'b1*', 'b#', '*', '*', '*', 'b3*', 'f3*', '*', 'w#', '*']),
        list(['d@', '*', 'w#', 'n@', 'r3', '*', 'f1', 'f1n#'])],
       [list([]), list([]), list([]), ..., list(['x@']), list([]),
        list([])],
       [list([]), list([]), list([]), ...,
        list(['f3', 'f1n#', 'f3*', 'n@', '*', '*', '*', 'f1w#', '*']),
        list(['m3', 'b1d@', 'b1', '*', 'b3', '*', 'b2', '*']),
        list(['n@', '*', 'n@', 'f1*', '*', 'f1', '*', '*', '*', '*', '*'])],
       ...,
       [list([]), list([]), list([]), ..., list([]), list([]), list([])],
       [list([]), list([]), list([]), ..., list([]), list([]), list([])],
       [list([]), list([]), list([]), ..., list([]), list([]), list([])]],
      dtype=object)

In [27]:
# Building up the model function

#################### The threshold on shots rarity ###################
threshold = 1                                                        #
######################################################################

# Given current prompt p_t and shot s_t, return next prompt p_(t+1)
def model(prompt, shot):
    n_grams_ps = Model[poss_prompts.index(prompt), all_shots.index(shot)]
    if(len(n_grams_ps) > threshold):    # exclude rare reactions
        return np.random.choice(n_grams_ps)
    return 'n@'   # when (p, s) never happened, return unforced error in net

# Example: RN just served ('5') and we want to reply with a forehand ('f28'). This is what RN will reply (stochastic function)
model('5', 'f28')

'f3'

# Your turn to serve!

You can play against the historical RN: just type in the shots and see if you can beat him! You can try first with simple shots, like 'f3' or 'b2' or even with a volley 'v1'.

In [8]:
current_prompt = 'X'
while True:       ###### CAREFUL WHEN INTERRUPTING
    shot = input()
    next_prompt = model(current_prompt, shot)
    print(next_prompt)
    current_prompt = next_prompt
    if is_win(next_prompt) == 1:
        print('Point won!')
        break
    if is_win(next_prompt) == 0:
        print('Point lost!')
        break

KeyboardInterrupt: 

# Example and tester

We provide here an example of policy and a function with which you can test your model.

For testing your personal new policy, simply modify the first line of code and insert your policy dictionary. 

The final scoring will be done with this same function, with number_of_points_played >= 1000000 and different values of the threshold, to be sure that statistical fluctuations of the result are reduced.

In [22]:
serves_opponent = [shot for shot in [sh[0] for sh in shots_opo_com] if shot[0] in '456']

['6',
 '5',
 '4',
 '6',
 '6',
 '5',
 '4',
 '6',
 '4',
 '4',
 '6',
 '4',
 '4',
 '4',
 '5',
 '6',
 '4',
 '4',
 '6',
 '4',
 '6',
 '5',
 '4',
 '6',
 '4',
 '6',
 '5',
 '5',
 '5',
 '4',
 '6',
 '5',
 '5',
 '4',
 '6',
 '5',
 '5',
 '6',
 '6',
 '5',
 '4',
 '4',
 '5',
 '4',
 '6',
 '4',
 '6',
 '5',
 '4',
 '5',
 '5',
 '4',
 '5',
 '6',
 '5',
 '6',
 '5',
 '5',
 '4',
 '5',
 '5',
 '5',
 '6',
 '6',
 '5',
 '5',
 '4',
 '4',
 '5',
 '5',
 '4',
 '4',
 '5',
 '5',
 '4',
 '5',
 '5',
 '6',
 '5',
 '4',
 '5',
 '4',
 '4',
 '4',
 '5',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '4',
 '5',
 '6',
 '4',
 '4',
 '5',
 '4',
 '4',
 '4',
 '5',
 '4',
 '4',
 '4',
 '4',
 '5',
 '6',
 '4',
 '4',
 '6',
 '6',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '6',
 '6',
 '5',
 '4',
 '6',
 '4',
 '6',
 '4',
 '4',
 '5',
 '5',
 '5',
 '6',
 '4',
 '6',
 '4',
 '6',
 '4',
 '6',
 '4',
 '6',
 '6',
 '4',
 '6',
 '4',
 '6',
 '5',
 '5',
 '5',
 '5',
 '5',
 '4',
 '4',
 '6',
 '4',
 '4',
 '5',
 '5',
 '4',
 '5',
 '4',
 '4',
 '4',
 '5',
 '5',
 '4',
 '4',
 '6',
 '4',
 '4'

In [113]:
# Test function that makes your policy (a.k.a. Phederos) play against historical RN !!

############################## Modify here! ##########################
#example_policy = {'f3': ['b2', 'b2', 'b3', 'v1'], 'f1': ['f1']}
#example_policy = fed_strat
example_policy = combined2
#example_policy = fed_strat_combined
######################################################################

number_of_points_played = 150000

serves_opponent = [shot for shot in [sh[0] for sh in shots_opo_com] if shot[0] in '456']

points_won  = 0
points_lost = 0


for num in range(number_of_points_played):
    print(num, end = '\r')
    # Random choice on who is serving
    if np.random.randint(0,2) == 0 :
        current_prompt = 'X'
    else :
        current_prompt = np.random.choice(serves_opponent)
    while True:
        shot = np.random.choice(example_policy.get(current_prompt,['5']))
        next_prompt = model(current_prompt, shot)
        current_prompt = next_prompt
        if is_win(current_prompt) == 1:
            points_won  += 1
            break
        if is_win(current_prompt) == 0:
            points_lost += 1
            break
            
print('How well are we performing? '     , points_won/(points_lost + points_won))
print('Baseline historical performance: ', 1 - (df['Winner'].sum()/len(df['Winner'])))

How well are we performing?  0.5998466666666666
Baseline historical performance:  0.5030959752321982


# Your turn now!
Show us that you are up for the challenge and make your Phederos destroy the historical RN! Just be careful and do not get carried away... real challenges lie ahead of you all!

In [52]:
#create the table that we said at the beginning

#df #['Shots']
#df['Server']
#for i in df['Shots', 'Server']
#    print(i)
#df['Shots', 'Server']
#df_i = df[['Shots','Server']]
all_shots = []
nadal = {}
for index,row in df.iterrows():
    server = row.Server
    outcome = row.Outcome
    winner = row.Winner
    shots = row.Shots.split(' ')
    if server == 1:  #Federer
        shots = ['x'] + shots
    
    
    for i in range(0,len(shots),2):
        try:
            all_shots.append([shots[i], shots[i+1], outcome, winner])
        except:
            all_shots.append([shots[i], 'Z', outcome, winner])

all_shots_df = pd.DataFrame(all_shots, columns=['Nadal','Federer', 'Outcome', 'Winner', 'Last'])
all_shots_df
print(shots)
    

['5', 's27', 'f+3', 'b2', 'v1', 'f-1']


In [16]:
#baseline strategy

nadal_strat = {}
federer_strat = {}
for index, row in df.iterrows():
    server = row.Server
    shots = row.Shots.split(' ')
    current = server
    winner = row.Winner
    if winner == 0:
        continue
    for i in range(len(shots)-1):
        if current == 0:  #Nadal
            if shots[i] not in nadal_strat.keys():
                if shots[i+1] in all_shots:
                    nadal_strat[shots[i]] = [shots[i+1]]
            else:
                if shots[i+1] in all_shots:
                    nadal_strat[shots[i]].append(shots[i+1])
            current = 1
        else:
            if shots[i] not in federer_strat.keys():
                if shots[i+1] in all_shots:
                    federer_strat[shots[i]] = [shots[i+1]]
            else:
                if shots[i+1] in all_shots:
                    federer_strat[shots[i]].append(shots[i+1])
            current = 0


'''
Allowed shots:  ['4', '4+', '5', '6', '6+', 'X', 'b', 'b1', 'b17', 'b18', 'b19', 'b2', 'b27', 
                 'b28', 'b29', 'b3', 'b37', 'b38', 'f', 'f+1', 'f+3', 'f1', 'f18', 'f2', 'f27', 
                 'f28', 'f29', 'f3', 'f37', 'f38', 'r2', 's', 's1', 's2', 's28', 's3', 'v1', 'v3', 'z1', 'z3']  
Possible prompts:  ['f3', 'f1', 'b2', 'b1', 'X', 'f2', 'b3', '4', '6', '5', '*', 'b28', 'f28', 'b27', 's2', 
'f+3', 's3', 'n@', 'f+1', 'b18', 'd@', 'f38', 'b29', 's1', 'b38', 'f37', '#', 'w@', 'f27', 'f29', 'n#', 'd#', 
'f', 'b', 's28', 'b37', 'b17', 'f1*', 'b19', 'b39', 'w#', 'r2', 'f18', 'f39', 'z3', 'v1', 'v3', 'f#', 's27', 
'4+', '6+', 'z1', 'b1*', 'f3*', 'm2', 'f17', 'r3', 's', 'r1', 's29', 's38', 'b+1', 'z2', 'o3', 'c6', 'v2', 'c4',
'b1w@', 'b#', 'b1n@', 'b+3', 'f19', 'b3*', 'f+2', 'f1w@', 'b3w@', 's37', 's39', 'f3n@', 'f3w@', 'm3', 'f1n@', 
'b1d@', 'b2n#', 'f2d@', 'f1n#', 'b1w#', 's#', 'b1n#', 'x@', 'f1w#', 'y3', 'f2d#', 'b2d@', 'f2n#', 'f3d@', 'y1', 
'b3w#', 'o1', 'f-3']
''' 
for i in poss_prompts:
    if i not in federer_strat.keys():
        if i[-1] in ['@','#','*']:
            federer_strat[i] = ['f1']
        elif i=='X':
            federer_strat[i] = ['f1']
            
#f_strat = federer_strat.copy()
#for i in f_strat.keys():
#    if i not in all_shots:
#        del federer_strat[i]
            
for i in poss_prompts:
    if i not in federer_strat.keys():
        print(i)

for i in federer_strat.keys():
    print('{}\t{}'.format(i, len(federer_strat[i])))

m3
b;37	1
b18	74
f1	689
b27	49
4	489
b2	338
f+1	121
5	173
s1	86
b28	147
6	268
f+3	75
f3	270
f2	276
b3	304
c6	9
f17	18
f18	32
f38	8
s2	37
f28	45
4+	51
z1	19
v3	15
f+2	12
6+	44
z3	15
f29	32
f;1	4
b9	3
f+	3
b	8
b19	61
v	1
s+1	8
s	4
b37	15
b8	2
f	3
f8	2
j3	2
f-1	2
5+	13
b1	229
s28	26
r2	4
s3	55
s18	4
f27	14
b;3	5
s39	6
m1	1
b+2	8
p2	2
s29	18
r+1	1
o2	2
c4	9
b29	50
o1	5
b+1	12
v1	25
b-3	2
r1	5
f3+3	1
i3	1
f19	16
b38	23
s38	9
o3	3
f1-	2
i2	5
o=3	1
b39	10
o=1	1
y2	1
j2	1
c5	4
u+1	5
y+1	9
b+3	6
s37	5
m2	1
b;28	1
b+28	1
cc6	1
s+2	1
y3	3
u+3	2
s17	1
r3	2
b17	11
v2	5
b+18	1
b;27	1
b;2	2
z2	7
s+3	1
y	1
s-2	1
s27	5
f37	3
f39	4
f-3	1
j1	1
b32	1
y1	4
i1	1
j+1	3
c4+	1
h+3	1
cc4	1
z+1	1
i=2	1
k3	1
r28	1
s-1	1
j=+1	1
o=2	1
f;3	2
y+3	1
j+2	1
s19	2
p3	1
f+29	1
b;17	1
h2	1
r29	2
X	1
*	1
n@	1
d@	1
#	1
w@	1
n#	1
d#	1
f1*	1
w#	1
f#	1
b1*	1
f3*	1
b1w@	1
b#	1
b1n@	1
b3*	1
f1w@	1
b3w@	1
f3n@	1
f3w@	1
f1n@	1
b1d@	1
b2n#	1
f2d@	1
f1n#	1
b1w#	1
s#	1
b1n#	1
x@	1
f1w#	1
f2d#	1
b2d@	1
f2n#	1
f3d@	1
b3w#	1


In [108]:
def build_federer_winning_df():
    df_f = df[df.Winner==1]

    all_lasts = []
    for index, row in df_f.iterrows():
        server = row.Server
        shots = row.Shots.split(' ')
        outcome = row.Outcome
        #print(server, shots, outcome)
        if outcome in ['*', 'w*']:  # Last shot federer
            if len(shots) >= 2:
                last_nadal = shots[-2]
                last_federer = shots[-1]
            else:
                #print(server, shots, outcome)
                continue
        else:
            if len(shots) >= 3:
                last_nadal = shots[-3]
                last_federer = shots[-2]
            else:
                #print(server, shots, outcome)
                continue
        all_lasts.append([last_nadal, last_federer])
    all_lasts_df = pd.DataFrame(all_lasts, columns=['Nadal', 'Federer'])

    return all_lasts_df

def build_nadal_winning_df():
    df_f = df[df.Winner==0]

    all_lasts = []
    for index, row in df_f.iterrows():
        server = row.Server
        shots = row.Shots.split(' ')
        outcome = row.Outcome
        #print(server, shots, outcome)
        if outcome in ['*', 'w*']:  # Last shot nadal
            if len(shots) >= 2:
                last_nadal = shots[-1]
                last_federer = shots[-2]
            else:
                #print(server, shots, outcome)
                continue
        else:
            if len(shots) >= 3:
                last_nadal = shots[-2]
                last_federer = shots[-3]
            else:
                #print(server, shots, outcome)
                continue
        all_lasts.append([last_nadal, last_federer])
    all_lasts_df = pd.DataFrame(all_lasts, columns=['Nadal', 'Federer'])

    return all_lasts_df


        
def build_federer_strategy(all_lasts_df):
    fed_strat = {}
    fed_strat_prob = {}
    for index, row in all_lasts_df.iterrows():
        key = row['Nadal']
        value = row['Federer']
        if value not in all_shots:
            continue            
        if key not in fed_strat.keys():
            fed_strat[key] = [value]
        else:
            fed_strat[key].append(value)
    
    # add needed prompts
    for i in poss_prompts:
        if i not in fed_strat.keys():
            if i[-1] in ['@','#','*']:
                fed_strat[i] = ['']
            elif i == 'X':   #Nadal waits for us to serve
                fed_strat[i] = ['6']
            else:
                fed_strat[i] = ['f1']
    
    for i in fed_strat.keys():
        #fed_strat_prob[i] = 
        aux = dict(Counter(fed_strat[i]))
        
        for j in aux.keys():
            aux[j] = round(aux[j] / len(fed_strat[i]),2)
        fed_strat_prob[i] = aux
    
    return fed_strat, fed_strat_prob

def build_nadal_strategy(all_lasts_df):
    nad_strat = {}
    nad_strat_prob = {}
    for index, row in all_lasts_df.iterrows():
        key = row['Federer']
        value = row['Nadal']
        if value not in all_shots:
            continue            
        if key not in nad_strat.keys():
            nad_strat[key] = [value]
        else:
            nad_strat[key].append(value)
    
    # add needed prompts
    for i in poss_prompts:
        if i not in nad_strat.keys():
            if i[-1] in ['@','#','*']:
                nad_strat[i] = ['']
            elif i == 'X':   #Nadal waits for us to serve
                nad_strat[i] = ['6']
            else:
                nad_strat[i] = ['f1']
    
    for i in nad_strat.keys():
        #fed_strat_prob[i] = 
        aux = dict(Counter(nad_strat[i]))
        
        for j in aux.keys():
            aux[j] = round(aux[j] / len(nad_strat[i]),2)
        nad_strat_prob[i] = aux
    
    return nad_strat, nad_strat_prob

federer_df = build_federer_winning_df()
nadal_df = build_nadal_winning_df()

#print(fed_strat)
#fed_strat_prob
#nadal_df


federer_df.Nadal = federer_df.Nadal.str.strip('c')
federer_df.Federer = federer_df.Federer.str.strip('c')
nadal_df.Nadal = nadal_df.Nadal.str.strip('c')
nadal_df.Federer = nadal_df.Federer.str.strip('c')


fed_strat, fed_strat_prob = build_federer_strategy(federer_df)
nad_strat, nad_strat_prob = build_nadal_strategy(nadal_df)


In [112]:
print('----')
combined = {}
for i in poss_prompts:
    combined[i] = {}
    for j in fed_strat_prob[i].keys():
        combined[i][j] = fed_strat_prob[i][j] - 2.0*(len(nadal_df[nadal_df.Nadal==j]) / len(nadal_df))
        

for i in poss_prompts:
    for j in combined[i].keys():
        if combined[i][j] < 0:
            combined[i][j] = 0
for i in poss_prompts:
    for j in combined[i].keys():

        #combined[i][j] /= sum(combined[i].values())
        combined[i][j] *= 100

        combined2 = {}
        
for i in combined.keys():
    combined2[i] = []
    for j in combined[i].keys():
        combined2[i].extend([j]*int(combined[i][j]))
combined2

----


{'f3': ['b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  's2',
  's2',
  's2',
  'z1',
  'z1',
  'z1',
  'z1',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's1',
  's1',
  's1',
  's1',
  'z3'],
 'f1': ['f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'f2'],
 'b2': ['f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'z3',
  'z3',
  'z3',
  'z3',
  'v3',
  'v3',
  'v3',
  'v1',
  'z1'],
 'b1': ['f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v1',
  'v1',
  'f+3'],
 'X': ['6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6

In [117]:
# check the resuts
for i in combined2.keys():
    print('{}\t{}'.format(i,len(combined2[i])))

f3	35
f1	24
b2	23
b1	19
X	100
f2	21
b3	41
4	69
6	72
5	67
*	100
b28	34
f28	33
b27	59
s2	72
f+3	53
s3	47
n@	100
f+1	31
b18	20
d@	100
f38	19
b29	9
s1	42
b38	6
f37	48
#	100
w@	100
f27	43
f29	48
n#	100
d#	100
f	95
b	95
s28	61
b37	39
b17	33
f1*	100
b19	25
b39	46
w#	100
r2	62
f18	21
f39	48
z3	74
v1	56
v3	74
f#	100
s27	74
4+	69
6+	69
z1	69
b1*	100
f3*	100
m2	69
f17	92
r3	50
s	69
r1	61
s29	69
s38	83
b+1	69
z2	98
o3	69
c6	69
v2	52
c4	69
b1w@	100
b#	100
b1n@	100
b+3	84
f19	69
b3*	100
f+2	87
f1w@	100
b3w@	100
s37	67
s39	46
f3n@	100
f3w@	100
m3	69
f1n@	100
b1d@	100
b2n#	100
f2d@	100
f1n#	100
b1w#	100
s#	100
b1n#	100
x@	100
f1w#	100
y3	74
f2d#	100
b2d@	100
f2n#	100
f3d@	100
y1	69
b3w#	100
o1	56
f-3	69


In [14]:
#create the table that we said at the beginning

def nadal_winning_serves():
    all_serves = []
    nadal = {}
    for index,row in df[df.Winner == 1].iterrows():
        server = row.Server
        outcome = row.Outcome
        winner = row.Winner
        shots = row.Shots.split(' ')
        if server == 1:  #Federer
            shots = ['X'] + shots

        try:
            all_serves.append([shots[0], shots[1]])
        except:
            all_serves.append([shots[0], 'Z'])
    
    all_serves_df = pd.DataFrame(all_serves, columns=['Nadal','Federer'])
    del all_serves
    return all_serves_df

def build_federer_winning_df():
    df_f = df[df.Winner==1]

    all_lasts = []
    for index, row in df_f.iterrows():
        server = row.Server
        shots = row.Shots.split(' ')
        outcome = row.Outcome
        #print(server, shots, outcome)
        if outcome in ['*', 'w*']:  # Last shot federer
            if len(shots) >= 4:
                last_nadal = shots[-2]
                last_federer = shots[-1]
            else:
                #print(server, shots, outcome)
                continue
        else:
            if len(shots) >= 5:
                last_nadal = shots[-3]
                last_federer = shots[-2]
            else:
                #print(server, shots, outcome)
                continue
        all_lasts.append([last_nadal, last_federer])
    all_lasts_df = pd.DataFrame(all_lasts, columns=['Nadal', 'Federer'])

    return all_lasts_df

In [23]:
all_serves_df = nadal_winning_serves()
all_endgame_df = build_federer_winning_df()
#combined_df = [all_serves_df, all_endgame_df]
combined_df = pd.concat([all_serves_df,all_endgame_df])

In [25]:
fed_strat_combined, fed_strat_prob = build_federer_strategy(combined_df)
fed_strat

{'5': ['b18',
  'b28',
  'f38',
  'f28',
  'f18',
  'f28',
  'f28',
  'b18',
  's28',
  's28',
  'b28',
  'b18',
  'f1',
  'b2',
  'b2',
  'f1',
  'b2',
  'f27',
  'b28',
  'f29',
  'b18',
  'b27',
  'b29',
  'f29',
  'b19',
  'f38',
  'f37',
  'b18',
  'b29',
  'f18',
  'b18',
  'b38',
  'b18',
  'b28',
  'b28',
  'b27',
  'b28',
  'b18',
  'b28',
  'f38',
  'b27',
  'f29',
  'f29',
  'b27',
  'b27',
  'f29',
  'b29',
  'f29',
  'f28',
  'b18',
  'f29',
  'b28',
  'b18',
  'f18',
  'b27',
  'b18',
  'b19',
  'b29',
  'f27',
  'b29',
  'f27',
  'b19',
  'b28',
  'f37',
  'b28',
  'b28',
  'f27',
  'b28',
  'f29',
  'b18',
  'f28',
  'b28',
  'b28',
  'b38',
  'b28',
  'f28',
  'b28',
  'f29',
  'b28',
  'b28',
  'f29',
  'b27',
  'b38',
  'b28',
  'f28',
  'b19',
  'f38',
  'f29',
  'b29',
  'f38',
  'b18',
  'f38',
  'b18',
  'b17',
  'f18',
  'b28',
  'b28',
  'b28',
  'f29',
  'b17',
  'b27',
  'f29',
  'b29',
  's28',
  'f27',
  'f29',
  'b18',
  'b28',
  'f28',
  'b27',
  'b18',
 